In [14]:
import sys
import logging
import json
from datetime import datetime, timezone

# -----------------------------
# JSON formatter (Loki / OpenSearch)
# -----------------------------
class JsonFormatter(logging.Formatter):
    def format(self, record: logging.LogRecord) -> str:
        log = {
            "ts": datetime.now(timezone.utc)
            .isoformat(timespec="milliseconds")
            .replace("+00:00", "Z"),
            "level": record.levelname,
            "logger": record.name,
            "msg": record.getMessage(),
        }
        for key, value in record.__dict__.items():
            if key.startswith("_") or key in (
                "name","msg","args","levelname","levelno","pathname","filename",
                "module","exc_info","exc_text","stack_info","lineno","funcName",
                "created","msecs","relativeCreated","thread","threadName",
                "processName","process"
            ):
                continue
            log[key] = value
        return json.dumps(log, ensure_ascii=False)

def setup_logging():
    logger = logging.getLogger("web3_ingestion")
    logger.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(JsonFormatter())
    if not logger.handlers:
        logger.addHandler(handler)
    logger.propagate = False
    return logger

log = setup_logging()

In [10]:
import os
import time
import random
import asyncio
from typing import NamedTuple, Optional
from aiohttp import ClientSession, ClientTimeout
from web3 import Web3
from web3.middleware import ExtraDataToPOAMiddleware
from src.metrics import RPC_REQUESTS, RPC_ERRORS, RPC_KEY_BUSY, ACTIVE_RPC_CONNECTIONS
from src.logging import log

In [16]:
# -----------------------------
# Exceptions
# -----------------------------
class RpcKeyUnavailable(Exception): pass
class RpcTemporarilyUnavailable(Exception): pass

In [26]:
# -----------------------------
# RPC Key Slot: 滑动窗口频率控制
# -----------------------------
class RpcKeySlot:
    def __init__(self, key_env: str, min_interval: float):
        self.key_env = key_env
        self.min_interval = min_interval
        self.last_used_at = 0.0
        self.lock = asyncio.Lock()

    async def try_acquire(self) -> bool:
        async with self.lock:
            now = time.time()
            if now - self.last_used_at < self.min_interval:
                return False
            self.last_used_at = now
            return True

In [31]:
# -----------------------------
# RPC Provider
# -----------------------------
class RpcProvider:
    def __init__(self, name, base_url, weight, key_env=None, key_interval=1.0):
        self.name = name
        self.base_url = base_url,
        self.key_env = key_env,
        self.base_weight = weight
        self.current_weight = weight
        self.cooldown_until = 0
        self.key_slots = []

        if key_env:
            if not isinstance(key_env, list):
                key_env = [key_env]
            self.key_slots = [RpcKeySlot(env, key_interval) for env in key_env]

        self._key_cursor = 0

    def available(self) -> bool:
        return time.time() >= self.cooldown_until

    def penalize(self, seconds=15):
        self.current_weight = max(1, self.current_weight - 1)
        self.cooldown_until = time.time() + seconds

    def reward(self):
        if self.current_weight < self.base_weight:
            self.current_weight += 1

    async def acquire_key(self, max_wait=2.0, sleep_step=0.05):
        if not self.key_slots:
            return self.base_url, None

        deadline = time.time() + max_wait
        while time.time() < deadline:
            for _ in range(len(self.key_slots)):
                slot = self.key_slots[self._key_cursor]
                self._key_cursor = (self._key_cursor + 1) % len(self.key_slots)
                if await slot.try_acquire():
                    api_key = os.getenv(slot.key_env)
                    if api_key:
                        return f"{self.base_url}/{api_key}", slot
            await asyncio.sleep(sleep_step)
        raise Exception(f"No available RPC key for provider={self.name}")

In [36]:
# -----------------------------
# RpcPool
# -----------------------------
class RpcPool:
    def __init__(self, providers):
        self.providers = providers

    def get_available_providers(self):
        candidates = []
        for p in self.providers:
            if p.available():
                candidates.extend([p] * p.current_weight)
        random.shuffle(candidates)
        return candidates
    
    @classmethod
    def from_config(cls, rpc_configs: dict, chain: str) -> "RpcPool":
        chain_cfg = rpc_configs.get("chains", {}).get(chain)
        if not chain_cfg:
            raise RuntimeError(f"Chain config not found: {chain}")

        providers = []
        for cfg in chain_cfg.get("providers", []):
            if not cfg.get("enabled", True):
                continue
            providers.append(RpcProvider(
                name=cfg["name"],
                base_url=cfg["base_url"],
                weight=int(cfg.get("weight", 1)),
                key_env=cfg.get("api_key_env"),
            ))

        if not providers:
            raise RuntimeError(f"No RPC providers enabled for chain: {chain}")

        for p in providers:
            log.info(
                "rpc_enabled",
                extra={
                    "chain": chain,
                    "rpc": p.name,
                    "key_envs": [k.key_env for k in p.key_slots] if p.key_slots else ["public"],
                    "weight": p.base_weight,
                },
            )

        return cls(providers)

In [29]:
# -----------------------------
# RpcContext
# -----------------------------
class RpcContext(NamedTuple):
    rpc: str
    key_env: str

In [49]:
# -----------------------------
# 异步滑动窗口 RPC Router
# -----------------------------
class Web3AsyncSlidingRouter:
    def __init__(self, rpc_pool: RpcPool, chain: str, timeout=10, penalize_seconds=15, max_backoff=30):
        self.rpc_pool = rpc_pool
        self.chain = chain
        self.timeout = timeout
        self.penalize_seconds = penalize_seconds
        self.max_backoff = max_backoff
        self.consecutive_failures = 0

    async def call(self, fn):
        last_exc = None
        providers = self.rpc_pool.get_available_providers()
        used = set()

        for provider in providers:
            if provider.name in used:
                continue
            used.add(provider.name)

            try:
                url, slot = await provider.acquire_key()
            except Exception:
                # RPC_KEY_BUSY.labels(chain=self.chain, rpc=provider.name).inc()
                continue

            key_env_label = slot.key_env if slot else "public"
            # RPC_REQUESTS.labels(chain=self.chain, rpc=provider.name, key_env=key_env_label).inc()
            # ACTIVE_RPC_CONNECTIONS.labels(chain=self.chain, rpc=provider.name).inc()

            # ⚠️ 使用同步 Web3，但通过 asyncio.to_thread() 异步执行
            w3 = Web3(Web3.HTTPProvider(url, request_kwargs={"timeout": self.timeout}))
            w3.middleware_onion.inject(ExtraDataToPOAMiddleware, layer=0)

            try:
                result = await asyncio.to_thread(fn, w3)
                provider.reward()
                self.consecutive_failures = 0
                return result, RpcContext(rpc=provider.name, key_env=key_env_label)

            except Exception as e:
                log.warning(
                    "⚠️ rpc_failover",
                    extra={
                        "chain": self.chain,
                        "rpc": provider.name,
                        "key_env": key_env_label,
                        "error": str(e)[:200],
                    },
                )
                # RPC_ERRORS.labels(chain=self.chain, rpc=provider.name, key_env=key_env_label).inc()
                provider.penalize(self.penalize_seconds)
                last_exc = e
            # finally:
                # ACTIVE_RPC_CONNECTIONS.labels(chain=self.chain, rpc=provider.name).dec()

        # 全部失败
        self.consecutive_failures += 1
        backoff = min(5 * self.consecutive_failures, self.max_backoff)
        await asyncio.sleep(backoff)
        raise Exception(f"RPC temporarily unavailable for chain={self.chain}") from last_exc


完整示例，展示如何用 Web3AsyncSlidingRouter 连续滑动发 RPC 请求，并发发送 100+ 个请求，模拟未来接 Kafka/Queue 场景

In [ ]:
import asyncio

RPC_CONFIG_PATH = "/etc/ingestion/rpc_providers.json" 
CHAIN = os.getenv("CHAIN", "bsc").lower()

with open(RPC_CONFIG_PATH) as f:
    rpc_configs = json.load(f)

In [ ]:
# -----------------------------
# Notebook 测试示例
# -----------------------------
async def get_latest_block(w3):
    return w3.eth.block_number

async def rpc_worker(router, worker_id, queue):
    while True:
        fn = await queue.get()
        try:
            result, ctx = await router.call(fn)
            print(f"[Worker-{worker_id}] {ctx.rpc}/{ctx.key_env} -> {result}")
        except Exception as e:
            print(f"[Worker-{worker_id}] RPC failed: {e}")
        finally:
            queue.task_done()

async def main():
    rpc_pool = RpcPool.from_config(rpc_configs, chain="bsc")
    router = Web3AsyncSlidingRouter(rpc_pool, chain="bsc", timeout=10)

    queue = asyncio.Queue()
    for _ in range(10):
        queue.put_nowait(get_latest_block)

    workers = [asyncio.create_task(rpc_worker(router, i, queue)) for i in range(10)]
    await queue.join()
    for w in workers:
        w.cancel()
    await asyncio.gather(*workers, return_exceptions=True)

In [54]:
await main()
# ankr/ANKR_KEY_A -> <coroutine object get_latest_block at 0x74fa74866140>

{"ts": "2026-01-21T15:34:22.923Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "quiknode", "key_envs": ["QUIKNODE_KEY"], "weight": 8}
{"ts": "2026-01-21T15:34:22.924Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "moralis", "key_envs": ["MORALIS_KEY_B"], "weight": 8}
{"ts": "2026-01-21T15:34:22.924Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "infura", "key_envs": ["INFURA_KEY_A", "INFURA_KEY_B", "INFURA_KEY_C"], "weight": 8}
{"ts": "2026-01-21T15:34:22.925Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "blockpi", "key_envs": ["BLOCKPI_BSC_KEY_A", "BLOCKPI_BSC_KEY_B", "BLOCKPI_BSC_KEY_C"], "weight": 8}
{"ts": "2026-01-21T15:34:22.925Z", "level": "INFO", "logger": "web3_ingestion", "msg": "rpc_enabled", "chain": "bsc", "rpc": "ankr", "key_envs": ["ANKR_KEY_A", "ANKR_KEY_B", "ANKR_KEY_C"], "weight": 6}
{"

🔹 说明

1. 异步滑动窗口

    - 每个 Web3AsyncSlidingRouter.call() 都是协程，非阻塞。
    - RpcKeySlot 控制每个 key 的最小间隔。

2. 并发 worker

    - rpc_worker 从 asyncio.Queue 拉取任务并发执行。
    - 可以根据吞吐量调整 worker 数量。

3. 连续滑动流

    - 队列里放 100 个 RPC 请求，Worker 会并行处理。
    - 没有 batch 概念，每个请求独立流式发送。

4. 超时处理

    - 超过 timeout=10s 会自动切换 provider/key。

5. Kafka/Queue 兼容

    - queue 可以替换成 Kafka Consumer/Producer。

    - result, ctx 可以直接写到 Kafka 或 DB。

6. 💡优化建议：
    - worker 数量可以根据 CPU / RPC provider 并发能力调节。
    - min_interval 可以针对不同 key 设置，避免 provider 限速。
    - 如果你有 Kafka 队列，可以把 queue.get() 换成 Kafka 消费消息流，实现真正 实时流式 RPC 调用 + commit。